# Week 4: Retrieval-Augmented Generation (RAG) with arXiv Papers
This week marks a major shift in your AI agent's capabilities: you’ll build the foundation for a Retrieval-Augmented Generation (RAG) system tailored to scientific research. Rather than relying on an LLM’s memory alone, RAG architectures allow your agent to search a structured knowledge base and generate grounded, document-aware answers.

Your task is to create a RAG pipeline using recent arXiv cs.CL papers, converting them into searchable chunks, embedding them, and indexing them with FAISS. You’ll then implement a simple query interface that takes a user question, retrieves the top relevant chunks, and displays them for further processing.

This week marks the beginning of building your agent’s private research knowledge base—a semantic index that you’ll evolve into a full-featured hybrid database in Week 5.


## 📚 Learning Objectives

* Understand the components of a Retriever-Reader QA pipeline.
* Explore document chunking strategies (e.g., sections vs. sliding windows) and their impact on retrieval performance.
* Index scientific text using vector embeddings and FAISS.
* Build and query a semantic index via a FastAPI endpoint that returns relevant passages.


## Project Design

The project will guide you through building a RAG pipeline on arXiv cs.CL papers:

1. **Data Collection:** Obtain 50 arXiv cs.CL PDFs (you can scrape via the arXiv API or use a provided sample set).
2. **Text Extraction:** Extract raw text from each PDF (for example, using PyMuPDF's `get_text()` on each page). Clean and concatenate the page text into full-document strings.
3. **Text Chunking:** Split each paper into chunks (≤ 512 tokens each). You might split at section boundaries or use a sliding-window approach (e.g., 500-token windows with overlap). Chunking into smaller, meaningful segments (around 250–512 tokens) often yields better retrieval precision.
4. **Embedding Generation:** Compute dense vector embeddings for each chunk. For instance, using the `sentence-transformers` library:

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(list_of_chunks)  # embeds each text chunk into a 384-d vecto

   (Alternatively, you can use a Hugging Face Transformer model and apply pooling manually to get chunk embeddings.)
5. **Indexing with FAISS:** Build a FAISS index of the chunk embeddings. For example, use a simple index like `IndexFlatL2` with the same dimensionality as your embeddings. Add all chunk vectors to the index (e.g., `index.add(np.array(embeddings))`).
6. **Notebook Demo:** Create a notebook where a user query is embedded and passed to the index (`index.search(query_embedding, k)`) to retrieve the top-3 matching chunks. Display the original chunk text for these results.
7. **FastAPI Service:** Build a simple FastAPI app. Define an endpoint (e.g. `@app.get("/search")`) that accepts a query parameter `q`. In the handler, embed `q`, perform the FAISS search, and return the top passages as JSON. (For example, a FastAPI endpoint can accept a question and return relevant documents.)



## Starter Code Snippets

Below are skeleton code templates. Fill in the details (indicated by comments or ellipses).

**Data Extraction (PDF → Text):**

In [ ]:
import fitz  # PyMuPDF
from typing import List
import urllib, urllib.request
import feedparser
import requests
from io import BytesIO
import ssl, certifi, urllib.request

context = ssl.create_default_context(cafile=certifi.where())

def ssl_read_url(url: str) -> str:
    return urllib.request.urlopen(url, context=context).read()

def get_pdf_urls() -> List[str]:
    url = f"https://export.arxiv.org/api/query?search_query=all:electron&start=0&max_results=50"
    data = ssl_read_url(url)
    res = feedparser.parse(data)
    return [
            link.href
            for entry in res.entries
            for link in entry.links
            if "pdf" in link.href
          ]

    
def extract_text_from_url(url: str) -> str:
    """
    Open a PDF and extract all text as a single string.
    """
    response = requests.get(url)

    pdf_bytes = BytesIO(response.content)
    doc = fitz.open(stream=pdf_bytes, filetype="pdf")
    pages = []
    for page in doc:
        page_text = page.get_text()  # get raw text from page
        # (Optional) clean page_text here (remove headers/footers)
        pages.append(page_text)
    full_text = "\n".join(pages)
    return full_text

In [11]:
urls = get_pdf_urls()
print(len(urls))

50


In [12]:
full_texts = [extract_text_from_url(url) for url in urls]
print(len(full_texts))

50


**Chunking Logic (Sliding Window):**

In [36]:
def chunk_text(text: str, max_tokens: int = 512, overlap: int = 50) -> List[str]:
    tokens = text.split()
    chunks = []
    step = max_tokens - overlap
    for i in range(0, len(tokens), step):
        chunk = tokens[i:i + max_tokens]
        chunks.append(" ".join(chunk))
    return chunks

chunks = [
        chunk  
        for text in full_texts 
        for chunk in chunk_text(text)
]

with open("chunks.json", "w", encoding="utf-8") as f:
    json.dump(chunks, f, ensure_ascii=False, indent=4)

In [28]:
print([chunk[100:] for chunk in chunks[-5:]])

['Wiley-Interscience Publication, p61. [11] E. Benedetto et al., Proceedings of PAC07, p. 4033 (2007).', 'Proceedings of PAC07, p. 4033 (2007).', 'parallel plates separated by 0.2 mm is depicted. The electric ﬁeld, driving the electron, 0 0.05 0.1', 'ctron and Positron Accelerators (MBI97), KEK, Tsukuba, Japan, 15-18 July 1997, KEK Proceedings 97-17', 'build up sim- ulations at CERN, in Proc.of ECLOUD12, La Biodola, Isola d’Elba, Italy, 5-9 June 2012.']



**Embedding Generation (Sentence-Transformers):**

In [30]:
from sentence_transformers import SentenceTransformer
from numpy import ndarray

model = SentenceTransformer("all-MiniLM-L6-v2")

def get_embeddings(chunks: List[str]) -> ndarray:
    return model.encode(chunks) 

embeddings = get_embeddings(chunks)


**FAISS Indexing and Search:**


In [37]:
import json
import faiss
import numpy as np

# Assume embeddings is a 2D numpy array of shape (num_chunks, dim)
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)  # using a simple L2 index
index.add(np.array(embeddings))  # add all chunk vectors

# Example: search for a query embedding
queries = ["How do electron holes in materials work",
           "Explain how spin-background affects low-lying excitations",
           "Explain the properties of metallic adatoms",
           "Explain hole-spin qubits",
           "Explain why it's hard to scale up quantum computers"
          ]
query_embeddings = model.encode(queries)  # get embedding for the query (shape: [1, dim])
k = 3
distances, indicesList = index.search(query_embeddings, k)
indicesList = indicesList.tolist()
retrieval_report = [
    {
        "request": queries[i],
        "response": [chunks[index] for index in indices]
    }
    for i, indices in enumerate(indicesList)
]

import pickle

with open("index.pkl", "wb") as f:
    pickle.dump(index, f)

with open("retrieval_report.json", "w", encoding="utf-8") as f:
    json.dump(retrieval_report, f, ensure_ascii=False, indent=4)



**FastAPI Route Skeleton:**


In [ ]:

from fastapi import FastAPI
import numpy as np

app = FastAPI()

@app.get("/search")
async def search(q: str):
    """
    Receive a query 'q', embed it, retrieve top-3 passages, and return them.
    """
    # TODO: Embed the query 'q' using your embedding model
    query_vector = ...  # e.g., model.encode([q])[0]
    # Perform FAISS search
    k = 3
    distances, indices = faiss_index.search(np.array([query_vector]), k)
    # Retrieve the corresponding chunks (assuming 'chunks' list and 'indices' shape [1, k])
    results = []
    for idx in indices[0]:
        results.append(chunks[idx])
    return {"query": q, "results": results}



## Deliverables

* **Code Notebook / Script:** Complete code for the RAG pipeline (PDF extraction, chunking, embedding, indexing, retrieval).
* **Data & Index:** The FAISS index file and the set of 50 processed paper chunks (e.g., as JSON or pickled objects).
* **Retrieval Report:** A brief report showing at least 5 example queries and the top-3 retrieved passages for each, to demonstrate system performance.
* **FastAPI Service:** The FastAPI app code (e.g. `main.py`) and instructions on how to run it. The `/search` endpoint should be demonstrable (e.g. returning top-3 passages in JSON for sample queries).

## Student Exploration Tips

* Experiment with different chunk sizes and overlaps. Smaller chunks (∼250 tokens) often give more precise retrieval, while larger chunks include more context.
* Try different embedding models (e.g. using `'all-mpnet-base-v2'` or `'paraphrase-MiniLM-L6-v2'`) to see how retrieval results change.
* Implement a simple reranking step: for example, after retrieving candidates with FAISS, re-score them with a cross-encoder model for finer ranking.
* Use metadata: consider filtering or weighting chunks by paper metadata (e.g. year, authors, keywords) to improve relevance if needed.
